# Miniproject-DQN Notebook
### Paul Boulenger, Paul Devianne

## Part 1: Introduction
### Question 1.a

In [ ]:

action = action = { # DO NOTHING
        'confinement': False, 
        'isolation': False, 
        'hospital': False, 
        'vaccinate': False,
    }

episode_length = 30 #30 weeks

for i in range(episode_length):
    env.step(action) 

## Part 2: Professor Russo's Policy

## Part 3: A Deep Q-learning approach

## Part4: Dealing with a more complex action Space

## Part5: Wrapping Up